In [2]:
#test to generate the hexagon system for the AP

import h3
import json
from shapely.geometry import Polygon
import geopandas as gpd
from shapely.ops import transform
import pyproj

# Define your coordinates (assuming these are boundary coordinates in the order: nw, ne, se, sw)
coordinates = [12.1111, 34.5, 32.0, 59.8]
nw, ne, sw, se = (coordinates[2], coordinates[1]), (coordinates[3], coordinates[1]), (coordinates[2], coordinates[0]), (coordinates[3], coordinates[0])

# Define the resolution
resolution = 6

# Create the bounding box polygon
ring = [nw, ne, se, sw, nw]
polygon = Polygon(ring)  # Create a shapely Polygon

# Generate H3 hexagons within the bounding box
hexagons = h3.polyfill_geojson(polygon.__geo_interface__, resolution)

# Prepare the hexagon geometries for GeoJSON
features = []
transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)

for hex in hexagons:
    hex_boundary = h3.h3_to_geo_boundary(hex, geo_json=True)
    # Convert hex_boundary into a format suitable for the GeoJSON (a list of lists of coordinate tuples)
    projected_boundary = [[transformer.transform(*point) for point in hex_boundary]]
    # Create a GeoJSON feature for each hexagon
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': projected_boundary
        },
        'properties': {
            'h3_index': hex
        }
    }
    features.append(feature)

# Construct the GeoJSON
geojson = {
    'type': 'FeatureCollection',
    'features': features,
    'crs': {
        'type': 'name',
        'properties': {
            'name': 'EPSG:3857'
        }
    }
}

# Save the GeoJSON to a file
output_directory = "Z:/Thesis/Data/H3"  # Change this to your actual directory path
output_filename = "h3_hexagons_APtest.geojson"
output_path = f"{output_directory}/{output_filename}"

with open(output_path, 'w') as f:
    json.dump(geojson, f)

# Note: The GeoJSON file now has the CRS for Web Mercator, but the H3 library generates coordinates in WGS 1984.
# The GeoJSON specification does not support coordinate transformation, so ensure that the data is correctly projected in your GIS software.


In [ ]:
! pip uninstall h3

In [ ]:
! pip install h3==3.7.0

In [1]:
! pip show h3

Name: h3
Version: 4.1.2
Summary: Uber's hierarchical hexagonal geospatial indexing system
Home-page: 
Author: 
Author-email: Uber Technologies <ajfriend@gmail.com>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whethe

In [12]:
# Produces H3 hexagons for the AP

import h3
import json
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj

# Define your coordinates
coordinates = [12, 40, 30, 75]
#[14.7233,31.8878,29.7145,62.0855]
#[12.1111, 34.5, 32.0, 60.1]
nw, ne, sw, se = (coordinates[2], coordinates[1]), (coordinates[3], coordinates[1]), (coordinates[2], coordinates[0]), (coordinates[3], coordinates[0])

# Create the bounding box polygon
ring = [nw, ne, se, sw, nw]
polygon = Polygon(ring)

# Atmospheric levels
atmospheric_levels = ['10', '50', '100', '200', '300', '500', '700', '850', '925', '1000', 'SFC'] #10', '50', '100', '200', '300', '500', '700', '850', '925', '1000', 'SFC'] 
#['SFC', '1000', '925', '850', '700', '500', '300', '200', '100', '50', '10']

# Define the output directory
output_directory = "Z:/Thesis/Data/H3"

transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)

for index, level in enumerate(atmospheric_levels):
    # Generate H3 hexagons at the current resolution
    hexagons = h3.polyfill_geojson(polygon.__geo_interface__, index)

    # Prepare the hexagon geometries for GeoJSON
    features = []
    for hex in hexagons:
        hex_boundary = h3.h3_to_geo_boundary(hex, geo_json=True)
        projected_boundary = [[transformer.transform(*point) for point in hex_boundary]]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': projected_boundary
            },
            'properties': {
                'h3_index': hex
            }
        }
        features.append(feature)

    # Construct the GeoJSON
    geojson = {
        'type': 'FeatureCollection',
        'features': features,
        'crs': {
            'type': 'name',
            'properties': {
                'name': 'EPSG:3857'
            }
        }
    }

    # Save the GeoJSON to a file
    output_filename = f"h3_hexagons_res_{index}_level_{level}.geojson"
    output_path = f"{output_directory}/{output_filename}"
    
    with open(output_path, 'w') as f:
        json.dump(geojson, f)

    print(f"Generated H3 resolution {index} for atmospheric level {level} and saved to {output_path}")


Generated H3 resolution 0 for atmospheric level 10 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_0_level_10.geojson
Generated H3 resolution 1 for atmospheric level 50 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_1_level_50.geojson
Generated H3 resolution 2 for atmospheric level 100 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_2_level_100.geojson
Generated H3 resolution 3 for atmospheric level 200 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_3_level_200.geojson
Generated H3 resolution 4 for atmospheric level 300 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_4_level_300.geojson
Generated H3 resolution 5 for atmospheric level 500 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_5_level_500.geojson
Generated H3 resolution 6 for atmospheric level 700 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_6_level_700.geojson
Generated H3 resolution 7 for atmospheric level 850 and saved to Z:/Thesis/Data/H3/h3_hexagons_res_7_level_850.geojson
Generated H3 resolution 8 for atmospheric level 925 

MemoryError: 